In [1]:
from openpyxl import load_workbook

# Excelファイルのロード(読み取り専用)
excel_path='../archive_wiki/D18/D18-2018.7.24.xlsx'
workbook = load_workbook(filename=excel_path, read_only=True)

# Excelのシート名一覧を表示
#print(workbook.sheetnames)
sheet=workbook[workbook.sheetnames[1]]
sent_dic=[]
for row in sheet[f'A2:A{sheet.max_row}']:
    for cell in row:
        sent_dic.append(cell.value)
    
# ロードしたExcelファイルを閉じる
print(sent_dic[:5])
workbook.close()
sent_dic=list(filter(lambda sent:sent is not None,sent_dic))
#感性語辞典のエクセルの感性語と,その感情データ3人分を合体させて保存

['哀', '愛', '相いれない', '哀感', '愛敬']


In [5]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [2]:
query=sent_dic[0]
for word in sent_dic[1:]:
    query = query+" OR "+word
#print(query)

In [3]:
# -*- coding: utf-8 -*-
import pandas as pd
from requests_oauthlib import OAuth1Session
import json
import datetime, time, sys
from abc import ABCMeta, abstractmethod
import csv

CK = '8rpn77nybFyN8dibhRInImaj1' # Consumer Key
CS = 'n9OeoinWyJFcWVbl1werzPxxipL1nNsuSwBcLCd9S99GEzczV2' # Consumer Secret
AT = '2652252630-ZsGmyv3uLmLklLSiaRMZeqCbuwd6y6meXcTbHjZ' # Access Token
AS = 'GR0PFD9B64XMSTgajkfelQA06QrpoHvQwImg5fduRtm2i' # Accesss Token Secert

In [20]:
class TweetsGetter(object):
    __metaclass__ = ABCMeta

    def __init__(self):
        self.session = OAuth1Session(CK, CS, AT, AS)

    @abstractmethod
    def specifyUrlAndParams(self, keyword):
        '''
        呼出し先 URL、パラメータを返す
        '''

    @abstractmethod
    def pickupTweet(self, res_text, includeRetweet):
        '''
        res_text からツイートを取り出し、配列にセットして返却
        '''

    @abstractmethod
    def getLimitContext(self, res_text):
        '''
        回数制限の情報を取得 （起動時）
        '''

    def collect(self, total = -1, onlyText = False, includeRetweet = False):
        '''
        ツイート取得を開始する
        '''

        #----------------
        # 回数制限を確認
        #----------------
        self.checkLimit()

        #----------------
        # URL、パラメータ
        #----------------
        url, params = self.specifyUrlAndParams()
        params['include_rts'] = str(includeRetweet).lower()
        # include_rts は statuses/user_timeline のパラメータ。search/tweets には無効

        #----------------
        # ツイート取得
        #----------------
        cnt = 0
        unavailableCnt = 0
        while True:
            res = self.session.get(url, params = params)
            if res.status_code == 503:
                # 503 : Service Unavailable
                if unavailableCnt > 10:
                    raise Exception('Twitter API error %d' % res.status_code)

                unavailableCnt += 1
                print ('Service Unavailable 503')
                self.waitUntilReset(time.mktime(datetime.datetime.now().timetuple()) + 30)
                continue

            unavailableCnt = 0

            if res.status_code != 200:
                raise Exception('Twitter API error %d' % res.status_code)

            tweets = self.pickupTweet(json.loads(res.text))
            if len(tweets) == 0:
                # len(tweets) != params['count'] としたいが
                # count は最大値らしいので判定に使えない。
                # ⇒  "== 0" にする
                # https://dev.twitter.com/discussions/7513
                break

            for tweet in tweets:
                if (('retweeted_status' in tweet) and (includeRetweet is False)):
                    pass
                else:
                    if onlyText is True:
                        yield tweet['text']
                    else:
                        yield tweet

                    cnt += 1
                    #if cnt % 100 == 0:
                        #print ('%d件 ' % cnt)

                    if total > 0 and cnt >= total:
                        return

            params['max_id'] = tweet['id'] - 1

            # ヘッダ確認 （回数制限）
            # X-Rate-Limit-Remaining が入ってないことが稀にあるのでチェック
            if ('X-Rate-Limit-Remaining' in res.headers and 'X-Rate-Limit-Reset' in res.headers):
                if (int(res.headers['X-Rate-Limit-Remaining']) == 0):
                    self.waitUntilReset(int(res.headers['X-Rate-Limit-Reset']))
                    self.checkLimit()
            else:
                print ('not found  -  X-Rate-Limit-Remaining or X-Rate-Limit-Reset')
                self.checkLimit()

    def checkLimit(self):
        '''
        回数制限を問合せ、アクセス可能になるまで wait する
        '''
        unavailableCnt = 0
        while True:
            url = "https://api.twitter.com/1.1/application/rate_limit_status.json"
            res = self.session.get(url)

            if res.status_code == 503:
                # 503 : Service Unavailable
                if unavailableCnt > 10:
                    raise Exception('Twitter API error %d' % res.status_code)

                unavailableCnt += 1
                print ('Service Unavailable 503')
                self.waitUntilReset(time.mktime(datetime.datetime.now().timetuple()) + 30)
                continue

            unavailableCnt = 0

            if res.status_code != 200:
                raise Exception('Twitter API error %d' % res.status_code)

            remaining, reset = self.getLimitContext(json.loads(res.text))
            if (remaining == 0):
                self.waitUntilReset(reset)
            else:
                break

    def waitUntilReset(self, reset):
        '''
        reset 時刻まで sleep
        '''
        seconds = reset - time.mktime(datetime.datetime.now().timetuple())
        seconds = max(seconds, 0)
        print ('\n     =====================')
        print ('     == waiting %d sec ==' % seconds)
        print ('     =====================')
        sys.stdout.flush()
        time.sleep(seconds + 10)  # 念のため + 10 秒

    @staticmethod
    def bySearch(keyword):
        return TweetsGetterBySearch(keyword)

    @staticmethod
    def byUser(screen_name):
        return TweetsGetterByUser(screen_name)



class TweetsGetterBySearch(TweetsGetter):
    '''
    キーワードでツイートを検索
    '''
    def __init__(self, keyword):
        super(TweetsGetterBySearch, self).__init__()
        self.keyword = keyword

    def specifyUrlAndParams(self):
        '''
        呼出し先 URL、パラメータを返す
        '''
        url = 'https://api.twitter.com/1.1/search/tweets.json'
        params = {'q':self.keyword, 'count':100}
        return url, params

    def pickupTweet(self, res_text):
        '''
        res_text からツイートを取り出し、配列にセットして返却
        '''
        results = []
        for tweet in res_text['statuses']:
            results.append(tweet)

        return results

    def getLimitContext(self, res_text):
        '''
        回数制限の情報を取得 （起動時）
        '''
        remaining = res_text['resources']['search']['/search/tweets']['remaining']
        reset     = res_text['resources']['search']['/search/tweets']['reset']

        return int(remaining), int(reset)


class TweetsGetterByUser(TweetsGetter):
    '''
    ユーザーを指定してツイートを取得
    '''
    def __init__(self, screen_name):
        super(TweetsGetterByUser, self).__init__()
        self.screen_name = screen_name

    def specifyUrlAndParams(self):
        '''
        呼出し先 URL、パラメータを返す
        '''
        url = 'https://api.twitter.com/1.1/statuses/user_timeline.json'
        params = {'screen_name':self.screen_name, 'count':200}
        return url, params

    def pickupTweet(self, res_text):
        '''
        res_text からツイートを取り出し、配列にセットして返却
        '''
        results = []
        for tweet in res_text:
            results.append(tweet)

        return results

    def getLimitContext(self, res_text):
        '''
        回数制限の情報を取得 （起動時）
        '''
        remaining = res_text['resources']['statuses']['/statuses/user_timeline']['remaining']
        reset     = res_text['resources']['statuses']['/statuses/user_timeline']['reset']

        return int(remaining), int(reset)


if __name__ == '__main__':

    # キーワードで取得
    for word in sent_dic:
        query=word
        for que_word in sent_dic[sent_dic.index(word)+1:]:
            if 220<len(query):
                break
            query = query+" OR "+que_word
        #print(len(query))
        getter = TweetsGetter.bySearch(query)

        # ユーザーを指定して取得 （screen_name）
        #getter = TweetsGetter.byUser('@realDonaldTrump')

        list_text = []

        for tweet in getter.collect(total = 10):
            #textの文字列を整形
            csvlinetext = tweet["text"].strip()            #ツイートデータから改行を削除
            csvlinetext = csvlinetext.replace(",", " ")   #カンマ文字を半角スペースに置換
            csvlines = csvlinetext.splitlines()           #配列変数に格納することで消しきれない改行を削除
            csvlinetext = ''.join(csvlines)
            list_text.append(csvlinetext)
        tweet_list={"text":list_text}
        tweet_list_df=pd.DataFrame(tweet_list)
        tweet_list_df.to_csv("./Tweetdata_utf-16.csv",encoding="utf-16",mode="a",header=False)

Exception: Twitter API error 429

In [26]:
df = pd.DataFrame(columns=['text'])
df_new = df.assign(text=list_text)

print(df_new[0:5])

                                                text
0  Get quality grades in:\n#Onlineclass\n#Homewor...
1  🎯ما نقدمه في #البرمجة😌\n#MachineLearning\n#Art...
2                      @Among_Natsu Pythonで組むパターンだ…！
3  We ensure #A+ in your 👔 🌒 \n #EssayHelp\n #Dat...
4  Check under that table if there's a python lyi...


In [13]:
print(list_text[1])

@unknown26842988 熱い応援メッセージありがとう✨

残念ながら、、、「ハズレ」です。
ごめんなさい。

🌟キャンペーン第3弾は10月18日～

🌟11月7日のチャレンジ当日は12時間生放送！視聴者限定キャンペーンも… https://t.co/VTTl4MgiaW
